In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import json

In [2]:
file_path = 'glass_y_sparse_cluster_50_clusters.json'
n_clusters = 50

# Read the data from the CSV file
df_data = pd.read_csv("glass_features.csv")

excluded = ["formula", "composition", "formula", "gfa"]
y = df_data['gfa'].values
X = df_data.drop(excluded, axis=1)

In [3]:
y.shape

(5680,)

In [4]:
import numpy as np
from scipy.stats import gaussian_kde

# Generate a sample one-dimensional vector
#np.random.seed(42)
#vector = np.random.randn(100)

# Estimate the density using kernel density estimation
kde = gaussian_kde(y)

# Evaluate the estimated density at sample points
sample_points = np.linspace(y.min(), y.max(), 100)
densities = kde(sample_points)

# Print the estimated density values
# for i in range(len(sample_points)):
#     print(f"Density at sample point {sample_points[i]}: {density[i]}")

In [5]:

density_dict = {}

# Print the density values for each sample position
for i in range(len(densities)):
    density = densities[i]
    #print(f"Density at sample position {i+1}: {density}")
    
    density_dict[i] = density

#print(density_dict)

sorted_density = sorted(density_dict.items(), key=lambda x: x[1])
#print(sorted_density)
top500 = [x[0] for x in sorted_density[:500]]
X_sparse = X.iloc[top500]
print(X_sparse.shape)

(100, 133)


In [6]:
id_map = {}

for i, index in enumerate(top500):
    id_map[i] = top500[i]
    

    

In [7]:
# Perform KMeans clustering
kmeans = KMeans(n_clusters=n_clusters, max_iter=500, random_state=222)
kmeans.fit(X_sparse)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

# Print cluster labels and centers
#print("Cluster Labels:", labels)
#print("Cluster Centers:", centers)

# Create a dictionary to store the cluster assignments
cluster = {}
#print(labels)
for i, cluster_id in enumerate(labels):
    if cluster_id in cluster:
        #cluster[int(cluster_id)].append(i)
        continue
    else:
        cluster[int(cluster_id)] = [id_map[i]]







/Users/jason/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [8]:
print(cluster)

{5: [49], 3: [48], 35: [50], 20: [47], 17: [51], 29: [46], 21: [52], 6: [45], 15: [44], 30: [54], 41: [56], 12: [41], 44: [59], 1: [36], 33: [62], 26: [63], 47: [34], 49: [64], 8: [65], 16: [31], 37: [67], 40: [29], 9: [27], 14: [71], 2: [24], 22: [73], 25: [74], 46: [22], 34: [21], 23: [17], 48: [14], 13: [82], 42: [13], 18: [12], 43: [9], 31: [7], 38: [87], 27: [4], 11: [88], 24: [1], 4: [0], 39: [90], 36: [91], 0: [92], 10: [94], 45: [95], 7: [96], 32: [97], 19: [98], 28: [99]}


In [9]:
selected_samples = []

# Print the size of each cluster
for key in sorted(cluster):
    #print(key, (cluster[key]))
    anchor_index = cluster[key][0]

    anchor_feature = X.iloc[anchor_index]
    #print(anchor_feature)
    distance_dict = {}
    for row_index in range(X.shape[0]):
        f = X.iloc[row_index]
        euclidean_distance = np.linalg.norm(anchor_feature - f)
        #print(euclidean_distance)
        
        if row_index == anchor_index:
            continue
        else:
            distance_dict[row_index] = euclidean_distance
    distance_dict = sorted(distance_dict.items(), key=lambda x: x[1])
    #print(distance_dict)
    n_added = 0
    for ck in distance_dict:
        if ck[0] not in selected_samples and n_added < 10:
            cluster[key].append(ck[0])
            selected_samples.append(ck[0])
            n_added += 1
    print(cluster[key])
    #break
for key in cluster:
    print(key, cluster[key])

[92, 123, 121, 93, 122, 127, 116, 113, 119, 118, 90]
[36, 35, 34, 17, 16, 63, 33, 15, 7, 29, 6]
[24, 20, 18, 38, 53, 50, 37, 52, 62, 25, 39]
[48, 72, 64, 73, 79, 88, 342, 360, 287, 262, 153]
[0, 483, 484, 500, 522, 528, 539, 545, 486, 551, 553]
[49, 60, 154, 67, 58, 46, 47, 159, 160, 161, 158]
[45, 44, 31, 61, 30, 22, 19, 23, 21, 42, 8]
[96, 135, 136, 99, 97, 134, 148, 142, 137, 141, 147]
[65, 55, 75, 77, 89, 51, 86, 74, 84, 40, 56]
[27, 3, 28, 13, 2, 4, 10, 9, 5, 59, 14]
[94, 130, 65, 157, 24, 78, 71, 111, 66, 85, 81]
[88, 48, 567, 617, 662, 663, 712, 664, 679, 721, 699]
[41, 32, 57, 54, 70, 43, 69, 26, 68, 76, 156]
[82, 83, 101, 109, 108, 110, 106, 87, 112, 102, 115]
[71, 80, 104, 176, 41, 168, 195, 91, 120, 244, 267]
[44, 45, 132, 129, 151, 150, 164, 268, 269, 264, 280]
[31, 265, 278, 285, 284, 311, 272, 296, 313, 337, 273]
[51, 274, 279, 126, 223, 301, 312, 295, 324, 323, 345]
[12, 11, 27, 36, 139, 138, 144, 149, 1, 96, 98]
[98, 146, 95, 145, 133, 100, 238, 237, 239, 254, 253]
[47,

In [10]:
# Example: Print formulas in cluster 0
for index in cluster[0]:
    print(df_data.iloc[index]["formula"])
    
# Save the dictionary to a file in JSON format
with open(file_path, 'w') as file:
    json.dump(cluster, file)

Al18FeCu
Al23FeNi
Al23Fe2
Al18FeNi
Al23FeCo
Al24Fe
Al22FeNi2
Al22Fe2Ni
Al23CoNi
Al23Co2
Al18CoNi
